In [1]:
import pandas as pd
import requests
# pip install beautifulsoup4
from bs4 import BeautifulSoup
import math
import numpy as np
# pip install cdifflib
import difflib
# pip install sys
import sys

In [2]:
# Set required dataset here.
data = pd.read_csv("data/users_training.csv")
data

,Unnamed: 0,id,name,username,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,...,description,created_at,class,tag_stock,tag_politics,tag_pronbot,tag_business,tag_fake_follower,tag_spambot,tag_traditional_spambot
0,0,418,Dennis Crowley,dens,69341,85422,2623,14990,4491,https://t.co/63fYABYs9J,...,"I like to build things (@Foursquare📱, @Stockad...",Wed Jul 05 19:52:46 +0000 2006,human,0,0,0,0,0,0,0
1,1,586,Chris Sacca,sacca,71148,1669241,1004,301974,12884,NaN,...,I love @crystale & our 3 girls. Used to invest...,Thu Jul 13 09:05:49 +0000 2006,human,0,0,0,0,0,0,0
2,2,8557,Japhy Grant,japhygrant,15191,3207,122,8901,145,https://t.co/jTIWv0ILjC,...,"👨‍🚀🌵Founder, #WonderValleyProjects. Had a hand...",Wed Oct 11 19:51:10 +0000 2006,human,0,0,0,0,0,0,0
3,3,12522,C.C. Chapman,cc_chapman,139668,47372,22593,12468,4370,NaN,...,Storyteller trying to leave it better than I f...,Wed Nov 15 15:03:14 +0000 2006,human,0,0,0,0,0,0,0
4,4,15913,Larry Hryb,majornelson,54804,1127687,5,7,11593,https://t.co/UNpCOu2HKp,...,"""The Xbox Guy"" 🎮 • 💑@thehappygirl • Twitter & ...",Wed Nov 22 20:28:03 +0000 2006,human,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19670,47841,4902726274,Max Jones,JayGShore,569,76,167,1948,1,NaN,...,#LFC,Fri Feb 12 21:26:12 +0000 2016,human,0,0,0,0,0,0,0
19671,47845,4903929447,Gagan Singh,gaganxsingh,2247,214,306,20973,1,https://t.co/Mqu3NYKTwr,...,your beautiful dark twisted fantasy.,Sat Feb 13 08:06:21 +0000 2016,human,0,0,0,0,0,0,0
19672,47847,4905375501,Dmitry Korotnian,finance_dk,623,27,180,108,8,NaN,...,"Financer, city man, assets manager",Sat Feb 13 17:33:30 +0000 2016,human,0,0,0,0,0,0,0
19673,47849,4908982317,Cifra73,cifra73,30978,437,510,27708,5,NaN,...,"Non smetto mai di avere paura, non smetto mai ...",Sun Feb 14 13:31:57 +0000 2016,human,0,0,0,0,0,0,0


In [3]:
# WEBSITE CRAWLER FOR TITLES

# FEATURE 1 (url_titles)
# FEATURE 2 (username-urltitle_similarity)
# FEATURE 3 (username_in_urltitle)



titles = []
usern_title_sim = []
usern_foundin_urlTitle = []

for idx, row in data.iterrows():
    # MAKE SURE THAT COLUMN NUMBER OF URL COLUMN MATCHES!!!!!!!
    if not pd.isnull(data.iloc[idx,9]):
       
        try:
            # Get the url (if 10 secs pass without establishing connection raise exception)
            # MAKE SURE THAT COLUMN NUMBER OF URL COLUMN MATCHES!!!!!!!
            r = requests.get(data.iloc[idx,9], timeout=10)
            # Get content of websites
            soup = BeautifulSoup(r.content)
            # Remove html tokens and append.
            titles.append(str(soup.title).replace("<title>" , "").replace("</title>", "").strip("|").strip())
            # If site allows access... else fill columns with nan.
            if titles[idx].strip() != "403 Forbidden":
                usern_title_sim.append(difflib.SequenceMatcher(None, data["username"][idx].replace("_", "").lower(),
                                                               titles[idx].replace("'", "").replace("_", "").lower()).ratio())
                usern_foundin_urlTitle.append(data["username"][idx].lower().replace("_", "") 
                                          in titles[idx].replace(" ", "").replace("'", "").replace("_", "").lower())
            else:
                usern_title_sim.append(math.nan)
                usern_foundin_urlTitle.append(math.nan)
        # Handle exception if trying to enter website exceeds limit of tries and return "Failed connection".       
        except Exception:
            titles.append("Failed connection")
            usern_title_sim.append(math.nan)
            usern_foundin_urlTitle.append(math.nan)
    else:
        titles.append(math.nan)
        usern_title_sim.append(math.nan)
        usern_foundin_urlTitle.append(math.nan)
     
    # Show progress in percentage.    
    sys.stdout.write("\r" + str(round((idx*100)/(len(data)-1), 1))+"%")
    sys.stdout.flush()
    

# Add features to dataset
data["url_titles"] = titles
data["username_urltitle_simimlarity"] = usern_title_sim
data["username_in_urltitle"] = usern_foundin_urlTitle

100.0%

In [4]:
# Similarity of username to screen name.
# FEATURE 4 (username_name_similarity)

usern_name_sim = []

for idx, row in data.iterrows():
    usern_name_sim.append(difflib.SequenceMatcher(None, str(data["username"][idx]).replace("_", "").lower()
                                                  , str(data["name"][idx])
                                                  .replace("'", "").replace("_", "").replace("-", "").lower()).ratio())
# Add feature to dataset
data["username_name_similarity"] = usern_name_sim

In [5]:
# username also featured in bio (the username must be an exact sub-set o th e description to match.)
# FEATURE 5 (username_in_bio)

username_found = []

for idx, row in data.iterrows():
    if data["username"][idx].replace("_", "").lower() in str(data["description"][idx]).lower().replace("'", "").replace("_", "").replace("-", ""):
        username_found.append(True)
    else:
        username_found.append(False)

data["username_in_bio"] = username_found

In [6]:
# lower user id, early twitter users 
# FEATURE 6 (lower_userid)

username_lower = []
# Find out what is considered a low id and set it as threshold.
id_threshold = 999999999 # 9 x9

for idx, row in data.iterrows():
    if data["id"][idx] <= id_threshold:
        username_lower.append(True)
    else:    
        username_lower.append(False)

# Add feature to dataset
data["lower_userid"] = username_lower

In [7]:
# followers to friends ratio (the bigger the ratio, the more the chance to be a business)
# FEATURE 7 (followers_friends_ratio)

# Add feature to dataset
data["followers_friends_ratio"] = data["followers_count"] / data["friends_count"]

In [8]:
# Bio Sentiment.
# FEATURE 8 (bio_sentiment)

# pip install afinn
from afinn import Afinn

sentiment_list = []
af = Afinn()

for idx, row in data.iterrows():
    if af.score(str(data["description"][idx])) > 0:
        sentiment_list.append("positive")
    elif af.score(str(data["description"][idx])) < 0:
        sentiment_list.append("negative")
    else:
        sentiment_list.append("neutral")
        

# Add feature to dataset
data["bio_sentiment"] = sentiment_list

In [9]:
# Show dataset with updated features.
final_df = data[["id",
                 "url_titles",
                 "username_urltitle_simimlarity",
                 "username_in_urltitle",
                 "username_name_similarity",
                 "username_in_bio",
                 "lower_userid",
                 "followers_friends_ratio",
                 "bio_sentiment"]].copy()

In [10]:
final_df

,id,url_titles,username_urltitle_simimlarity,username_in_urltitle,username_name_similarity,username_in_bio,lower_userid,followers_friends_ratio,bio_sentiment
0,418,Dennis Crowley,0.444444,False,0.444444,False,True,32.566527,positive
1,586,NaN,NaN,NaN,0.625000,False,True,1662.590637,positive
2,8557,Wonder Valley Projects (@wondervalleyprojects)...,0.162791,False,0.952381,False,True,26.286885,neutral
3,12522,NaN,NaN,NaN,0.857143,False,True,2.096756,positive
4,15913,Larry Hryb (@majornelson) • Instagram photos a...,0.333333,True,0.190476,True,True,225537.400000,neutral
...,...,...,...,...,...,...,...,...,...
19670,4902726274,NaN,NaN,NaN,0.222222,False,False,0.455090,neutral
19671,4903929447,Login • Instagram,0.285714,False,0.909091,False,False,0.699346,positive
19672,4905375501,NaN,NaN,NaN,0.320000,False,False,0.150000,positive
19673,4908982317,NaN,NaN,NaN,1.000000,False,False,0.856863,neutral


In [11]:
# Save new feautres into csv
final_df.to_csv("business_users_training.csv")
# Save new features but include the rest of the dataset aswell
data.to_csv("all_business_users_training.csv")